### Setup
Fine-tuning Llama-3 8B to solve problems from mathematical olympiads (AMC 10, AMC 12, AIME, etc.)

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-7j499enc/unsloth_10d4b8729b0a4c52b4f88f090625a403
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-7j499enc/unsloth_10d4b8729b0a4c52b4f88f090625a403
  Resolved https://github.com/unslothai/unsloth.git to commit 36488b58677a2283de55aed0461df0237bb88fb8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

### Dataset
We use the MATH dataset from [hendryks](https://huggingface.co/datasets/hendrycks/competition_math). This dataset consists of problems from mathematics competitions, including the AMC 10, AMC 12, AIME, and more. Each problem in MATH has a full step-by-step solution, which can be used to teach models to generate answer derivations and explanations.

In [ ]:
math_prompt = """Write a solution to the following math problem.

### Problem:
{}

### Solution:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatter(examples):
    problems  = examples["problem"]
    solutions = examples["solution"]
    texts = []
    for problem, solution in zip(problems, solutions):
        text = math_prompt.format(problem, solution) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("hendrycks/competition_math", split = "train")
dataset = dataset.map(formatter, batched = True,)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

### Model Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.143700
2,1.165800
3,1.189400
4,1.079700
5,1.158300
6,1.043900
7,0.895000
8,1.011600
9,0.915000
10,0.874300


### Inference
Test the model with a competition math problem.

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    math_prompt.format(
        "If $x + \frac{1}{x} = 3$, determine the value of $x^2 + \frac{1}{x^2}$.",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Write a solution to the following math problem.

### Problem:
If $x + rac{1}{x} = 3$, determine the value of $x^2 + rac{1}{x^2}$.

### Solution:
We have $x^2 + rac{1}{x^2} = (x + rac{1}{x})^2 - 2 = 9 - 2 = 7$.<|end_of_text|>
